In [55]:
import psycopg2

pg_host = "localhost"
pg_database = "graph"
pg_user = "postgres"
pg_password = "postgres"

# Establish PostgreSQL connection
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password
)
pg_cursor = pg_conn.cursor()

In [56]:
pg_cursor.execute("SELECT id, sentence from datasets")
datasets = pg_cursor.fetchall()
map_sentence_to_id = {sentence: id for id, sentence in datasets}

In [50]:
import os
import json
import glob
import pandas as pd

# Get all JSON files matching the pattern
json_files = glob.glob('output/labeling/graph*.json')

# Read each JSON file
data_list = []
for file_path in json_files:
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            data_list.extend(data)
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")

empty_df = []
for data in data_list:
    for option in data['option']:
        empty_df.append({
            'source_id': map_sentence_to_id[data['sentence']], 
            'sentence_1': data['sentence'], 
            'option_id': map_sentence_to_id[option['sentence']], 
            'sentence_2': option['sentence'], 
            'is_similar': option['is_similar']})
        
df = pd.DataFrame(empty_df)


In [45]:
df.head()

,source_id,sentence_1,option_id,sentence_2,is_similar
0,80064e85-095f-43b6-8366-407db5120af6,The system must verify that the Sales Person i...,6220c6dc-14fd-4d60-ad29-69c73a617c5e,The system must verify that the Sales Person o...,None
1,80064e85-095f-43b6-8366-407db5120af6,The system must verify that the Sales Person i...,32617737-6d40-41fe-ba11-bad6bd7b8990,The system must verify that the Sales Manager ...,True
2,80064e85-095f-43b6-8366-407db5120af6,The system must verify that the Sales Person i...,38cf7f2a-6542-4d04-a2cd-87fae6e4a77e,The system must verify that the Sales Person o...,None
3,80064e85-095f-43b6-8366-407db5120af6,The system must verify that the Sales Person i...,43162524-9b6e-4528-9c3f-ff2c7a52f647,The system must ensure that the Sales Person o...,None
4,80064e85-095f-43b6-8366-407db5120af6,The system must verify that the Sales Person i...,93e93241-0bc3-4955-bfb9-1c88590e6825,The system must ensure that the Sales Manager ...,None


In [51]:
for index, row in df.iterrows():
    is_similar = False if row['is_similar'] == None else True
    pg_cursor.execute("UPDATE result_option_graph SET is_similar = %s WHERE method = 'COSINE' and source_id = %s and option_id = %s ", (is_similar, row['source_id'], row['option_id']))
    pg_conn.commit()

In [57]:
import os
import json
import glob
import pandas as pd

# Get all JSON files matching the pattern
json_files = glob.glob('output/labeling/llm*.json')

# Read each JSON file
data_list = []
for file_path in json_files:
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            data_list.extend(data)
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")

empty_df = []
for data in data_list:
    for option in data['option']:
        empty_df.append({
            'source_id': map_sentence_to_id[data['sentence']], 
            'sentence_1': data['sentence'], 
            'option_id': map_sentence_to_id[option['sentence']], 
            'sentence_2': option['sentence'], 
            'is_similar': option['is_similar']})
        
df = pd.DataFrame(empty_df)


In [58]:
df.head()

,source_id,sentence_1,option_id,sentence_2,is_similar
0,a3425b00-0066-44cb-8caf-28f6b18d943b,Manage your just-in-time inventory (JIT) with ...,85b9338c-0e0b-4922-8afe-edb9454ba980,"Keep track of stock, incoming orders and produ...",None
1,a3425b00-0066-44cb-8caf-28f6b18d943b,Manage your just-in-time inventory (JIT) with ...,6780ed33-f1c3-4e7e-bce7-288a6410dfd3,"Trigger purchases, inter-warehouse transfers, ...",None
2,a3425b00-0066-44cb-8caf-28f6b18d943b,Manage your just-in-time inventory (JIT) with ...,df3f21c3-52c2-4bd3-a819-3dd292fd38aa,"Forecasted report of on-hand, incoming, and ou...",None
3,a3425b00-0066-44cb-8caf-28f6b18d943b,Manage your just-in-time inventory (JIT) with ...,478eab77-7ebc-45f7-a4b4-2ad7c5b06632,Notify procurement managers about low stock an...,None
4,a3425b00-0066-44cb-8caf-28f6b18d943b,Manage your just-in-time inventory (JIT) with ...,640fa6ac-1654-4e56-b886-2082f8fef7e3,Dedicated dashboards to keep the order fulfill...,None


In [59]:
for index, row in df.iterrows():
    is_similar = False if row['is_similar'] == None else True
    pg_cursor.execute("UPDATE result_option_llm SET is_similar = %s WHERE method = 'EUCLIDEAN' and source_id = %s and option_id = %s ", (is_similar, row['source_id'], row['option_id']))
    pg_conn.commit()